In [1]:
#loading data
import pandas as pd

dataset = pd.read_csv('housing.csv')

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
#viewing data
print(dataset.shape)
dataset[:5]

(506, 13)


,crim,zn,indus,chas,nox,age,rm,dis,rad,tax,ptratio,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,5.33,36.2


In [3]:
# Move 'medv' column to front, because it is the order required by Sagemaker
dataset = pd.concat([dataset['medv'], dataset.drop(['medv'], axis=1)], axis=1)

In [4]:
#Splitting data
from sklearn.model_selection import train_test_split
training_dataset, validation_dataset = train_test_split(dataset, test_size=0.1)

print(training_dataset.shape)
print(validation_dataset.shape)

(455, 13)
(51, 13)


In [5]:
training_dataset.to_csv('training_dataset.csv', index=False, header=False)
validation_dataset.to_csv('validation_dataset.csv', index=False, header=False)

In [6]:
#Uploading data to Amazon S3
import sagemaker

print(sagemaker.__version__)

sess = sagemaker.Session()
bucket = sess.default_bucket()

prefix = 'boston-housing'
training_data_path = sess.upload_data(path='training_dataset.csv', key_prefix=prefix + '/input/training')
validation_data_path = sess.upload_data(path='validation_dataset.csv', key_prefix=prefix + '/input/validation')

print(training_data_path)
print(validation_data_path)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
2.199.0
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
s3://sagemaker-us-east-1-726881595027/boston-housing/input/training/training_dataset.csv
s3://sagemaker-us-east-1-726881595027/boston-housing/input/validation/validation_dataset.csv


In [9]:
#Pulling XGBoost Docker image from Sagemaker
import boto3
from sagemaker import image_uris

region = boto3.Session().region_name    
container = image_uris.retrieve('xgboost', region, version="latest")

print(container)
#Creating the xgboost estimator with parameters specified
from sagemaker.estimator import Estimator

role = sagemaker.get_execution_role() 

ll_estimator = Estimator(container,
    role=role, 
    instance_count=1,
    instance_type='ml.m5.large',
    output_path='s3://{}/{}/output'.format(bucket, prefix)
)
ll_estimator.set_hyperparameters(num_round=50, max_depth=5, eta=0.2)

811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [10]:
#Creating data channels that will be accessed remotely bt the container
training_data_channel   = sagemaker.TrainingInput(s3_data=training_data_path, content_type='text/csv')
validation_data_channel = sagemaker.TrainingInput(s3_data=validation_data_path, content_type='text/csv')

ll_data = {'train': training_data_channel, 'validation': validation_data_channel}

In [11]:
#Training the model
ll_estimator.fit(ll_data)

INFO:sagemaker:Creating training-job with name: xgboost-2023-12-09-23-43-39-656


2023-12-09 23:43:39 Starting - Starting the training job...
2023-12-09 23:43:55 Starting - Preparing the instances for training.........
2023-12-09 23:45:18 Downloading - Downloading input data...
2023-12-09 23:46:03 Training - Downloading the training image...
2023-12-09 23:46:39 Training - Training image download completed. Training in progress..Arguments: train
[2023-12-09:23:46:42:INFO] Running standalone xgboost training.
[2023-12-09:23:46:42:INFO] File size need to be processed in the node: 0.03mb. Available memory size in the node: 335.66mb
[2023-12-09:23:46:42:INFO] Determined delimiter of CSV input is ','
[23:46:42] S3DistributionType set as FullyReplicated
[23:46:42] 455x12 matrix with 5460 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2023-12-09:23:46:42:INFO] Determined delimiter of CSV input is ','
[23:46:42] S3DistributionType set as FullyReplicated
[23:46:42] 51x12 matrix with 612 entries loaded from /opt/ml/input/data/validation?for

In [12]:
#Taking a look at the model artifacts created
%%bash -s "$ll_estimator.output_path"
aws s3 ls --recursive $1

2023-12-09 23:47:30          0 boston-housing/output/xgboost-2023-12-09-23-43-39-656/debug-output/training_job_end.ts
2023-12-09 23:47:29      30751 boston-housing/output/xgboost-2023-12-09-23-43-39-656/output/model.tar.gz
2023-12-09 23:47:30          0 boston-housing/output/xgboost-2023-12-09-23-43-39-656/profiler-output/framework/training_job_end.ts
2023-12-09 23:47:01      85970 boston-housing/output/xgboost-2023-12-09-23-43-39-656/profiler-output/system/incremental/2023120923/1702165500.algo-1.json
2023-12-09 23:47:00     183903 boston-housing/output/xgboost-2023-12-09-23-43-39-656/profiler-output/system/incremental/2023120923/1702165560.algo-1.json
2023-12-09 23:47:25      76626 boston-housing/output/xgboost-2023-12-09-23-43-39-656/profiler-output/system/incremental/2023120923/1702165620.algo-1.json
2023-12-09 23:47:30          0 boston-housing/output/xgboost-2023-12-09-23-43-39-656/profiler-output/system/training_job_end.ts


In [15]:
#Creating a Sagemaker endpoint
from time import strftime, gmtime
timestamp = strftime('%d-%H-%M-%S', gmtime())

endpoint_name = 'xgboost-demo-'+timestamp
print(endpoint_name)

xgboost-demo-09-23-48-55


In [22]:
#Deploying the model
ll_predictor = ll_estimator.deploy(endpoint_name=endpoint_name, 
                        initial_instance_count=1, 
                        instance_type='ml.t2.medium')

INFO:sagemaker:Creating model with name: xgboost-2023-12-10-00-14-06-415
INFO:sagemaker:Creating endpoint-config with name xgboost-demo-09-23-48-55
INFO:sagemaker:Creating endpoint with name xgboost-demo-09-23-48-55


------!

In [17]:
#Testing the model regression on  sample data
test_sample = '0.00632,18.00,2.310,0,0.5380,6.5750,65.20,4.0900,1,296.0,15.30,4.98'

In [18]:
ll_predictor.serializer = sagemaker.serializers.CSVSerializer()
ll_predictor.deserializer = sagemaker.deserializers.CSVDeserializer()

response = ll_predictor.predict(test_sample)
print(response)

[['24.230379104614258']]


In [19]:
#Testing on a list of samples
test_samples = ['0.00632,18.00,2.310,0,0.5380,6.5750,65.20,4.0900,1,296.0,15.30,4.98',
                '0.02731,0.00,7.070,0,0.4690,6.4210,78.90,4.9671,2,242.0,17.80,9.14']

response = ll_predictor.predict(test_samples)
print(response)
print(ll_predictor.endpoint_name)

[['24.230379104614258', '21.522525787353516']]
xgboost-demo-09-23-48-55


In [20]:
#Testing using the endpoint call
runtime = boto3.Session().client(service_name='runtime.sagemaker') 

response = runtime.invoke_endpoint(EndpointName=endpoint_name, 
                                  ContentType='text/csv', 
                                  Body=test_sample)

print(response['Body'].read())

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


b'24.230379104614258'


In [23]:
#Deleting the endpoint to free resources
ll_predictor.delete_endpoint()

INFO:sagemaker:Deleting endpoint configuration with name: xgboost-demo-09-23-48-55
INFO:sagemaker:Deleting endpoint with name: xgboost-demo-09-23-48-55
